# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import numpy as np;
import pandas as pd;

Next, import `Orders.csv` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("data/Orders.csv", index_col=0)
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# SUB PROBLEM 1: Find the total amount spent for each customer
customers_df = orders.groupby('CustomerID').sum()['amount_spent'].reset_index()

In [4]:
# SUB PROBLEM 2: Find the amount_spent values that correspond to quantiles 75% and 95%
Q95 = customers_df['amount_spent'].quantile(0.95)
Q75 = customers_df['amount_spent'].quantile(0.75)

In [5]:
# SUB PROBLEM 3: Create a funtion to label the customers according to their amount_spent values  
def check_customer_group(row):
    if row['amount_spent'] >= Q95:
        return 'VIP'
    elif row['amount_spent'] >= Q75 and row['amount_spent'] < Q95:
        return 'Preferred'
    return 'Regular'

In [6]:
# SUB PROBLEM 3: Apply the function to the dataframe
customers_df['Group'] = customers_df.apply(lambda row: check_customer_group(row), axis=1)

In [7]:
# SOLUTION
customers_group = customers_df[['CustomerID','Group']]
customers_group.head()

,CustomerID,Group
0,12346,VIP
1,12347,Preferred
2,12348,Preferred
3,12349,Preferred
4,12350,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

In [8]:
# SUB PROBLEM 1: Merge orders and customers_group_df to have 
# the country and customer group available in a single df
orders_customer = pd.merge(orders, customers_group, on='CustomerID', how='left')

In [9]:
# SUB PROBLEM 2: For each customer, find from which countries they made orders
columns = ['CustomerID','Country','Group']
country_customer = pd.DataFrame(orders_customer.groupby(columns).groups.keys(), columns=columns)

In [10]:
# SUB PROBLEM 3: For all customers, find those who are VIP.
country_vips = country_customer[country_customer['Group'] == 'VIP']

In [11]:
# SUB PROBLEM 4: Count the number of customers that made orders from each country
country_count = country_vips[['Country', 'CustomerID']].groupby('Country').count()

In [12]:
# SUB PROBLEM 5: Find the country with the largest number of VIP customers
country_count.nlargest(1,'CustomerID')

,CustomerID
Country,
United Kingdom,177


# Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [13]:
# SUB PROBLEM 1: For all customers, find those who are VIP or Preferred.
country_vip_pref = country_customer[country_customer['Group'].isin(['VIP','Preferred'])]

In [14]:
# SUB PROBLEM 2: Count the number of customers that made orders from each country
country_count = country_vip_pref[['Country', 'CustomerID']].groupby('Country').count()

In [15]:
# SUB PROBLEM 3: Find the country with the largest number of VIP and Preferred customers
country_count.nlargest(1,'CustomerID')

,CustomerID
Country,
United Kingdom,932
